In [ ]:
from azure.ai.ml import command, Input, Output, MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.entities import (
    Environment, Model, ManagedOnlineEndpoint, ManagedOnlineDeployment, CodeConfiguration
)
import uuid

# Authentification
credential = DefaultAzureCredential()
ml_client = MLClient(
    credential=credential,
    subscription_id="4eb48753-2226-4b34-8df1-ff473a98c57b",
    resource_group_name="rg-terrabyte0904",
    workspace_name="wrkspace-terrabyte0904"
)

# Données d'entrée
input_csv = Input(
    type="uri_file",
    path="azureml://subscriptions/4eb48753-2226-4b34-8df1-ff473a98c57b/resourcegroups/rg-terrabyte0904/workspaces/wrkspace-terrabyte0904/datastores/workspaceblobstore/paths/npk_predict_need.csv"
)

# Environnement
env = Environment(
    name="env-terrabyte",
    description="Environnement pour modèle NPK",
    conda_file="env_terrabyte.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:latest"
)
ml_client.environments.create_or_update(env)

# Récupération version
env = ml_client.environments.get(name="env-terrabyte", label="latest")

# Définir le job de training
job = command(
    code="./src",
    command="python process.py --input_data ${{inputs.input_data}} --registered_model_name terrabyte_model",
    inputs={"input_data": input_csv},
    outputs={"model_output": Output(type="uri_folder", mode="rw_mount")},
    compute="computej-terrabyte0904",
    environment=env.id,
    experiment_name="npk_exploration"
)

returned_job = ml_client.jobs.create_or_update(job)

In [ ]:
model = ml_client.models.create_or_update(
    Model(
        path=returned_job.outputs["model_output"].path,
        name="terrabyte_model",
        description="Modèle pour prédiction des besoins NPK",
        type="mlflow_model"
    )
)

print(returned_job.status)

# Création endpoint
endpoint_name = "terrabyte-endpoint-" + str(uuid.uuid4())[:8]

endpoint = ManagedOnlineEndpoint(
    name=endpoint_name,
    description="Endpoint pour prédiction des besoins NPK",
    auth_mode="key"
)

ml_client.begin_create_or_update(endpoint).result()

deployment = ManagedOnlineDeployment(
    name="blue",
    endpoint_name=endpoint.name,
    model=model.id,
    environment=env.id,
    instance_type="Standard_F8s_v2",
    instance_count=1,
    code_configuration=CodeConfiguration(
        code="./",  # Où se trouve score.py
        scoring_script="score.py"
    )
)

ml_client.online_deployments.begin_create_or_update(deployment).result()

# Logs
logs = ml_client.online_deployments.get_logs(
    name="blue",
    endpoint_name=endpoint.name,    
    lines=100
)
print(logs)

# Diriger tout le trafic vers le déploiement
endpoint.traffic = {"blue": 100}
ml_client.begin_create_or_update(endpoint).result()

print(f"✅ Endpoint déployé avec succès: {endpoint.name}")